Title: RP- Spatial Accessibility of COVID-19 Healthcare Resources in Illinois Pre-Processing Script
---

This is a script that automates the data gathering and pre-processing for our reproduction of Kang et al.

**Reproduction of**: Rapidly measuring spatial accessibility of COVID-19 healthcare resources: a case study of Illinois, USA

Original study *by* Kang, J. Y., A. Michels, F. Lyu, Shaohua Wang, N. Agbodo, V. L. Freeman, and Shaowen Wang. 2020. Rapidly measuring spatial accessibility of COVID-19 healthcare resources: a case study of Illinois, USA. International Journal of Health Geographics 19 (1):1–17. DOI:[10.1186/s12942-020-00229-x](https://ij-healthgeographics.biomedcentral.com/articles/10.1186/s12942-020-00229-x).

Reproduction Authors: Joe Holler, Kufre Udoh, Derrick Burt, Drew An-Pham, & Spring '21 Middlebury Geog 0323.

Reproduction Materials Available at: [RP-Kang Repository](https://github.com/derrickburt/RP-Kang-Improvements)

Created: `29 Jun 2021`
Revised: `29 Jun 2021`

### Modules
Import necessary libraries to run this model.
See `requirements.txt` for the library versions used for this analysis.

In [ ]:
# Load modules
import numpy as np
import pandas as pd
import geopandas as gpd
import networkx as nx
import osmnx as ox
from shapely.geometry import Point, LineString, Polygon
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import folium
import itertools
import os
import time
import warnings
from IPython.display import display, clear_output

warnings.filterwarnings("ignore")

## Check Directories

Because we have restructured the repository for replication, we need to check our working directory and make necessary adjustments.

In [ ]:
os.getcwd()

In [ ]:
## Use to set work directory properly
if os.getcwd() == '/home/jovyan/work/RP-Kang2020/procedure/code':
    os.chdir('../../')
if os.getcwd() == '/home/jovyan/work/RP-Kang2020/':
    None 

os.getcwd()

### Load and Plot the Street Network

In [ ]:
%%time
if not os.path.exists("data/raw/private/Chicago_Network_Buffer.graphml"):
    G = ox.graph_from_place('Chicago', network_type='drive', buffer_dist = 24140.2) # pulling the drive network the first time will take a while
    ox.save_graphml(G, 'raw/private/Chicago_Network_Buffer.graphml')
else:
    G = ox.load_graphml('raw/private/Chicago_Network_Buffer.graphml', node_type=str)
ox.plot_graph(G)

In [ ]:
# Get unique counts for each road network
# Turn nodes and edges in geodataframes
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

# Count
print(edges['maxspeed'].value_counts())

### Automate/Pre-Process Census Data  with API

*Note* you will need to download a new module called 'censusdata'

To do this, open a terminal in the cybergisx environment and type:

```pip install censusdata```

**Note: we deviate from the original paper's methodology here bringing in a larger buffer distance of census tracts

#### Tract

In [ ]:
# Load module
import censusdata as cd

In [ ]:
%time

# Read in all Illinois tracts using census API
pop_api = cd.download('acs5', 2018,
                             cd.censusgeo([('state', '17'), ('tract', '*')]),
                             ['B01001_001E', 'B01001_016E', 'B01001_017E', 'B01001_018E', 'B01001_019E', 
                              'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 
                              'B01001_025E', 'B01001_040E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 
                              'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E',
                              'B01001_049E'])

# Check
# pop_api.head()

In [ ]:
## Reformat and Rename columns
# Sum + Rename 50+ population
pop_api['OverFifty'] = pop_api.iloc[:, 1:21].sum(axis=1)

# Rename Total 
pop_api['TotalPop'] = pop_api['B01001_001E']

# Drop irrelevant columns
pop_api = pop_api.drop(pop_api.columns[0:21], axis=1)

# Check
pop_api.head()

In [ ]:
# ADD CODE TO SAFE INTO RAW DATA FOLDER
# Create column from index tract # -- we will need thee tract ID for a join
pop_api['TRACTCE'] = pop_api.index

# Convert to string 
pop_api['TRACTCE'] = pop_api['TRACTCE'].astype(str)

# Slice last 6 digits (tract id)
pop_api['TRACTCE'] = pop_api['TRACTCE'].str.slice(-6)

# Check
pop_api.head()

In [ ]:
### Note: We are using a larger subset of data here
len(pop_api)

#### Zip Codes

In [ ]:
# Read in all Illinois tracts using census API
zip_api = cd.download('acs5', 2019,
                             cd.censusgeo([('state', '17'), ('zip code tabulation area', '*')]),
                             ['B01003_001E'])

# check
zip_api.head()

In [ ]:
# rename population column
pop_col = {"B01003_001E":"pop"}
zip_api = zip_api.rename(columns=pop_col)

# Create column from index tract # -- we will need thee tract ID for a join
zip_api['ZCTA5CE10'] = zip_api.index

# Convert to string 
zip_api['ZCTA5CE10'] = zip_api['ZCTA5CE10'].astype(str)

# Slice last 6 digits (tract id)
zip_api['ZCTA5CE10'] = zip_api['ZCTA5CE10'].str.slice(6,11)

# Check
zip_api.head()

In [ ]:
len(zip_api)

### Automate/Pre-Process COVID-19 with Requests

Download covid data that will be kjoined to zip code geographies.

In [ ]:
import requests
import geojson
import json

In [ ]:
# Unfortunately... I have not found how to access archived COVID-119 case data, so this data is cases from 4/6/2021 - 6/30/2021 (or current date...) 
# Set file path
fp_covid = 'https://idph.illinois.gov/DPHPublicInformation/api/COVIDExport/GetZip'

# Make reqeuest
r_covid = requests.get(fp_covid)

# Save request as dataframe
covid_cases = pd.DataFrame.from_dict(json.loads(r_covid.content))

# Check
covid_cases.head()

In [ ]:
# Change confirmed cases to cases
cases_col = {'zip':"ZCTA5CE10", "confirmed_cases":"cases"}
covid_cases = covid_cases.rename(columns=cases_col)

In [ ]:
# Merge covid case data with zip code population to normalize cases
covid_api = covid_cases.merge(zip_api, how="inner", on="ZCTA5CE10")
covid_api

### Pull Census Boundary Shapefiles with FTP Site and Join to Population/Covid Case Data

#### Note: Here, we extract *census tracts* and *zip code geographies* based on their spatial relationship (intersection) with the street network

Census TIGER/Line shapefiles can bee accessed from ftp://ftp2.census.gov/geo/tiger/ using !wget

File path for Cook County 2010 tracts: ftp://ftp2.census.gov/geo/tiger//TIGER2010/TRACT/2010/tl_2010_17031_tract10.zip

File path for Illinois 2010 tracts: ftp://ftp2.census.gov/geo/tiger//TIGER2018/TRACT/tl_2018_17_tract.zip

#### Tracts

In [ ]:
# Check directory -- we want to downlaod the raw data directly into our pre-processing data folder
%ls

In [ ]:
# Download census tract shapefiles to data/raw/public/Pre-Processing/ for All of Chicago (017)
if not os.path.exists('data/raw/public/Pre-Processing/tl_2018_17_tract.zip'):
    !wget -P data/raw/public/Pre-Processing/ ftp://ftp2.census.gov/geo/tiger//TIGER2018/TRACT/tl_2018_17_tract.zip
    # Extract shapefiles
    !unzip -d data/raw/public/Pre-Processing/ data/raw/public/Pre-Processing/tl_2018_17_tract.zip
    # Read in all census tracts for Illinois
    tracts_shp = gpd.read_file('data/raw/public/Pre-Processing/tl_2018_17_tract.shp')
else:
    # Read in all census tracts for Illinois
    tracts_shp = gpd.read_file('data/raw/public/Pre-Processing/tl_2018_17_tract.shp')

In [ ]:
# Set crs to WGS 84
tracts_shp = tracts_shp.to_crs(epsg=4326)

# Select only tracts from Cook countiy + its adjacent counties
tracts_shp = tracts_shp.loc[(tracts_shp["COUNTYFP"] == '031') |
                            (tracts_shp["COUNTYFP"] == '043') |
                            (tracts_shp["COUNTYFP"] == '097') |
                            (tracts_shp["COUNTYFP"] == '197') ]

# Check crs
print(tracts_shp.crs)

# Check length
print(len(tracts_shp))

# Check column names
tracts_shp.head()

In [ ]:
# Rename columns for join
new_names = {"GEOID10":"GEOID", "TRACTCE10":"TRACTCE"}
tracts_shp = tracts_shp.rename(columns=new_names)

In [ ]:
# Join Tracts shape with Tracts Population data
## This drops duplicate values so that we do not end up with
atrisk_data = tracts_shp.merge(pop_api.drop_duplicates(subset=['TRACTCE']), how='inner', on="TRACTCE")
atrisk_data= atrisk_data.drop(atrisk_data.columns[5:10], axis=1)
len(atrisk_data)

In [ ]:
atrisk_data.head()

##### Zip codes

In [ ]:
# check directory -- we want to downlaod the raw tract data directly into our data folder
%ls

In [ ]:
# Download zip code shapefiles to data/raw/public/Pre-Processing/ for entire US
## I have not yet found a way to select by state before extracting
if not os.path.exists('data/raw/public/Pre-Processing/cb_2018_us_zcta510_500k.zip'):
    !wget -P data/raw/public/Pre-Processing/ ftp://ftp2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_zcta510_500k.zip
    # Extract shapefiles
    !unzip -d data/raw/public/Pre-Processing/ data/raw/public/Pre-Processing/cb_2018_us_zcta510_500k.zip
    # read in zip code data
    usa_zip = gpd.read_file('data/raw/public/Pre-Processing/cb_2018_us_zcta510_500k.shp')
else:
    # read in zip code data
    usa_zip = gpd.read_file('data/raw/public/Pre-Processing/cb_2018_us_zcta510_500k.shp')

In [ ]:
# Select only Illinois
ill_zip = usa_zip.loc[(usa_zip['GEOID10'] >= '60002') & (usa_zip['GEOID10'] <= '60827')]

# Set crs to WGS 84
ill_zip = ill_zip.to_crs(epsg=4326)

# Check crs
print(ill_zip.crs)

# Check length
print(len(ill_zip))

# Rename column names
ill_zip.head()

In [ ]:
# Join covid_zip to zip code geographis 
covid_zip_geo = ill_zip.merge(covid_api, how='inner', on='ZCTA5CE10')
# Drop extra columns
# covid_zip_geo = covid_zip_geo.drop(covid_zip_geo.columns[5:10], axis=1)
print(len(covid_zip_geo))
covid_zip_geo.head()

### Load Chicago Shapfile and Create Hexagon Grids (500-meter resolution)

commented out because have not figured how to manually create hex grids? (Should I delete?)

In [ ]:
# # Download Illinous plac tract shapefiles to data/raw/public/Pre-Processing/ for All of Illinious (017)
# if not os.path.exists('data/raw/public/Pre-Processing/tl_2020_17_place.zip'):
#     !wget -P data/raw/public/Pre-Processing/ ftp://ftp2.census.gov/geo/tiger//TIGER2020/PLACE/tl_2020_17_place.zip
#     # Extract shapefiles
#     !unzip -d data/raw/public/Pre-Processing/ data/raw/public/Pre-Processing/tl_2020_17_place.zip
#     # Read in all place shapefiles for Illinois
#     place_shp = gpd.read_file('data/raw/public/Pre-Processing/tl_2020_17_place.shp')
#     # Select only Chicago
#     chicago_shp = place_shp.loc[place_shp['NAME']=='Chicago']
#     # Save as shapefile
#     chicago_shp.to_file('data/raw/public/Pre-Processing/chicago_place.shp')
# else:
#     # Read in all census tracts for Illinois
#     chicago_shp = gpd.read_file('data/raw/public/Pre-Processing/chicago_place.shp')

In [ ]:
# chicago_shp.plot()

In [ ]:
# Read in and plot grid file for Chicago
grid_file = gpd.read_file('./data/raw/public/GridFile/Chicago_Grid.shp')
grid_file.plot(figsize=(8,8))

In [ ]:
grid_file.crs

### Hospital Data

Note that 999 is treated as a "NULL"/"NA" so these hospitals are filtered out. This data contains the number of ICU beds and ventilators at each hospital.

In [ ]:
hospitals = gpd.read_file('./data/raw/public/HospitalData/Chicago_Hospital_Info.shp')
hospitals.head()

### Automate/Pre-Process Hospital with Requests -- Still Drafting this

documentation for requests: https://docs.python-requests.org/en/master/

commented out because not working (should I delete?)

In [ ]:
# # ADD CODE TO SAFE INTO RAW DATA FOLDER
# # Set file paths for general care and icu beds
# fp_gen = 'https://opendata.arcgis.com/datasets/6ac5e325468c4cb9b905f1728d6fbf0f_0.geojson'
# fp_icu = 'https://healthdata.gov/resource/uqq2-txqb.json'

# # Eequests for g eneral care and icu beds
# r_gen = requests.get(fp_gen)
# r_icu = requests.get(fp_icu)

# # Get hospitals 
# hospitals_gen = gpd.GeoDataFrame.from_features(geojson.loads(r_gen.content),  crs="EPSG:26971")
# hospitals_icu = pd.DataFrame.from_dict(json.loads(r_icu.content))

# # # Filter for icu and general care
# hospitals_gen = hospitals_gen.loc[(hospitals_gen['STATE'] == 'IL') & (hospitals_gen['TYPE'] == 'GENERAL ACUTE CARE')]
# # ERROR Here: it is only taking the first thousand
# hospitals_icu = hospitals_icu[['hospital_pk', 'collection_week', 'state', 'city', 'ccn', 'hospital_name', 'zip', 'ccn', 'address', 'total_icu_beds_7_day_avg']]

# # Capitalize join column 
# hospitals_icu.rename(columns={'address':'ADDRESS'}, inplace=True)

# # Join
# hospitals_api = hospitals_gen.merge(hospitals_icu, on='ADDRESS')

# # Check 
# hospitals_icu['city'].unique()

## "Helper" Functions

The functions below are needed for our analysis later, let's take a look!

### network_setting

Cleans the OSMNX network to work better with drive-time analysis.

First, we remove all nodes with 0 outdegree because any hospital assigned to such a node would be unreachable from everywhere. Next, we remove small (under 10 node) *strongly connected components* to reduce erroneously small ego-centric networks. Lastly, we ensure that the max speed is set and in the correct units before calculating time.

Args:

* network: OSMNX network for the spatial extent of interest

Returns:

* OSMNX network: cleaned OSMNX network for the spatial extent

In [ ]:
def network_setting(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg ==0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg ==0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component)<10:
            for node in component:
                _nodes_removed+=1
                network.remove_node(node)
    for u, v, k, data in tqdm(G.edges(data=True, keys=True),position=0):
        if 'maxspeed' in data.keys():
            speed_type = type(data['maxspeed'])
            if (speed_type==str):
                # Add in try/except blocks to catch maxspeed formats that don't fit Kang et al's cases
                try:
                    if len(data['maxspeed'].split(','))==2:
                        data['maxspeed_fix']=float(data['maxspeed'].split(',')[0])                  
                    elif data['maxspeed']=='signals':
                        data['maxspeed_fix']=30.0 # drive speed setting as 35 miles
                    else:
                        data['maxspeed_fix']=float(data['maxspeed'].split()[0])
                except:
                    data['maxspeed_fix']=30.0 #miles
            else:
                try:
                    data['maxspeed_fix']=float(data['maxspeed'][0].split()[0])
                except:
                    data['maxspeed_fix']=30.0 #miles
        else:
            data['maxspeed_fix']=30.0 #miles
        data['maxspeed_meters'] = data['maxspeed_fix']*26.8223 # convert mile to meter
        data['time'] = float(data['length'])/ data['maxspeed_meters']
    print("Removed {} nodes ({:2.4f}%) from the OSMNX network".format(_nodes_removed, _nodes_removed/float(network.number_of_nodes())))
    print("Number of nodes: {}".format(network.number_of_nodes()))
    print("Number of edges: {}".format(network.number_of_edges()))    
    return(network)

### hospital_setting

Finds the nearest OSMNX node for each hospital.

Args:

* hospital: GeoDataFrame of hospitals
* G: OSMNX network

Returns:

* GeoDataFrame of hospitals with info on nearest OSMNX node

In [ ]:
def hospital_setting(hospitals, G):
    # Create an empty column 
    hospitals['nearest_osm']=None
    # Append the neaerest osm column with each hospitals neaerest osm node
    for i in tqdm(hospitals.index, desc="Find the nearest osm from hospitals", position=0):
        hospitals['nearest_osm'][i] = ox.get_nearest_node(G, [hospitals['Y'][i], hospitals['X'][i]], method='euclidean') # find the nearest node from hospital location
    print ('hospital setting is done')
    return(hospitals)

### pop_centroid

Converts geodata to centroids

Args:

* pop_data: a GeodataFrame
* pop_type: a string, either "pop" for general population or "covid" for COVID-19 case data

Returns:

* GeoDataFrame of centroids with population data

In [ ]:
def pop_centroid (pop_data, pop_type):
    pop_data = pop_data.to_crs({'init': 'epsg:4326'})
    # If pop is selected in dropdown, select at risk pop where population is greater than 0
    if pop_type =="pop":
        pop_data=pop_data[pop_data['OverFifty']>=0]
    # If covid is selected in dropdown, select where covid cases are greater than 0
    if pop_type =="covid":
        pop_data=pop_data[pop_data['cases']>=0]
    pop_cent = pop_data.centroid # it make the polygon to the point without any other information
    # Convert to gdf
    pop_centroid = gpd.GeoDataFrame()
    i = 0
    for point in tqdm(pop_cent, desc='Pop Centroid File Setting', position=0):
        if pop_type== "pop":
            pop = pop_data.iloc[i]['OverFifty']
            code = pop_data.iloc[i]['GEOID']
        if pop_type =="covid":
            pop = pop_data.iloc[i]['cases']
            code = pop_data.iloc[i].ZCTA5CE10
        pop_centroid = pop_centroid.append({'code':code,'pop': pop,'geometry': point}, ignore_index=True)
        i = i+1
    return(pop_centroid)

### djikstra_cca_polygons

Function written by Joe Holler + Derrick Burt. It is a more efficient way to calculate distance-weighted catchment areas for each hospital. The algorithm runs quicker than the original one ("calculate_catchment_area"). It first creaets a dictionary (with a node and its corresponding drive time from the hospital) of all nodes within a 30 minute drive time (using single_cource_dijkstra_path_length function). From here, two more dictionaries are constructed by querying the original one. From this dictionaries, single part convex hulls are created for each drive time interval and appended into a single list (one list with 3 polygon geometries). Within the list, the polygons are differenced from each other to produce three catchment areas.

Args:
* G: cleaned network graph *with node point geometries attached*
* nearest_osm: A unique nearest node ID calculated for a single hospital
* distances: 3 distances (in drive time) to calculate catchment areas from
* distance_unit: unit to calculate (time)

Returns:
* A list of 3 diffrenced (not-overlapping) catchment area polygons (10 min poly, 20 min poly, 30 min poly)

In [ ]:
def dijkstra_cca_polygons(G, nearest_osm, distances, distance_unit = "time"):
    
    '''
    
    Before running: must assign point geometries to street nodes
    
    # create point geometries for the entire graph
    for node, data in G.nodes(data=True):
    data['geometry']=Point(data['x'], data['y'])
    
    '''
    
    ## CREATE DICTIONARIES
    # create dictionary of nearest nodes
    nearest_nodes_30 = nx.single_source_dijkstra_path_length(G, nearest_osm, distances[2], distance_unit) # creating the largest graph from which 10 and 20 minute drive times can be extracted from
    
    # extract values within 20 and 10 (respectively) minutes drive times
    nearest_nodes_20 = dict()
    nearest_nodes_10 = dict()
    for key, value in nearest_nodes_30.items():
        if value <= 20:
            nearest_nodes_20[key] = value
        if value <= 10:
            nearest_nodes_10[key] = value
    
    ## CREATE POLYGONS FOR 3 DISTANCE CATEGORIES (10 min, 20 min, 30 min)
    # 30 MIN
    # If the graph already has a geometry attribute with point data,
    # this line will create a GeoPandas GeoDataFrame from the nearest_nodes_30 dictionary
    points_30 = gpd.GeoDataFrame(gpd.GeoSeries(nx.get_node_attributes(G.subgraph(nearest_nodes_30), 'geometry')))

    # This line converts the nearest_nodes_30 dictionary into a Pandas data frame and joins it to points
    # left_index=True and right_index=True are options for merge() to join on the index values
    points_30 = points_30.merge(pd.Series(nearest_nodes_30).to_frame(), left_index=True, right_index=True)

    # Re-name the columns and set the geodataframe geometry to the geometry column
    points_30 = points_30.rename(columns={'0_x':'geometry','0_y':'z'}).set_geometry('geometry')

    # Create a convex hull polygon from the points
    polygon_30 = gpd.GeoDataFrame(gpd.GeoSeries(points_30.unary_union.convex_hull))
    polygon_30 = polygon_30.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    # 20 MIN
    # Select nodes less than or equal to 20
    points_20 = points_30.query("z <= 20")
    
    # Create a convex hull polygon from the points
    polygon_20 = gpd.GeoDataFrame(gpd.GeoSeries(points_20.unary_union.convex_hull))
    polygon_20 = polygon_20.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    # 10 MIN
    # Select nodes less than or equal to 10
    points_10 = points_30.query("z <= 10")
    
    # Create a convex hull polygon from the points
    polygon_10 = gpd.GeoDataFrame(gpd.GeoSeries(points_10.unary_union.convex_hull))
    polygon_10 = polygon_10.rename(columns={0:'geometry'}).set_geometry('geometry')
    
    # Create empty list and append polygons
    polygons = []
    
    # Append
    polygons.append(polygon_10)
    polygons.append(polygon_20)
    polygons.append(polygon_30)
    
    # Clip the overlapping distance ploygons (create two donuts + hole)
    for i in reversed(range(1, len(distances))):
        polygons[i] = gpd.overlay(polygons[i], polygons[i-1], how="difference")

    return polygons

### hospital_measure_acc (adjusted to incorporate dijkstra_cca_polygons)

Measures the effect of a single hospital on the surrounding area. (Uses `dijkstra_cca_polygons`)

Args:

* \_thread\_id: int used to keep track of which thread this is
* hospital: Geopandas dataframe with information on a hospital
* pop_data: Geopandas dataframe with population data
* distances: Distances in time to calculate accessibility for
* weights: how to weight the different travel distances

Returns:

* Tuple containing:
    * Int (\_thread\_id)
    * GeoDataFrame of catchment areas with key stats

In [ ]:
def hospital_measure_acc (_thread_id, hospital, pop_data, distances, weights):
    # Create polygons
    polygons = dijkstra_cca_polygons(G, hospital['nearest_osm'], distances)
    
    # Calculate accessibility measurements
    num_pops = []
    for j in pop_data.index:
        point = pop_data['geometry'][j]
        # Multiply polygons by weights
        for k in range(len(polygons)):
            if len(polygons[k]) > 0: # To exclude the weirdo (convex hull is not polygon)
                if (point.within(polygons[k].iloc[0]["geometry"])):
                    num_pops.append(pop_data['pop'][j]*weights[k])  
    total_pop = sum(num_pops)
    for i in range(len(distances)):
        polygons[i]['time']=distances[i]
        polygons[i]['total_pop']=total_pop
        polygons[i]['hospital_icu_beds'] = float(hospital['Adult ICU'])/polygons[i]['total_pop'] # proportion of # of beds over pops in 10 mins
        polygons[i]['hospital_vents'] = float(hospital['Total Vent'])/polygons[i]['total_pop'] # proportion of # of beds over pops in 10 mins
        polygons[i].crs = { 'init' : 'epsg:4326'}
        polygons[i] = polygons[i].to_crs({'init':'epsg:32616'})
    print('\rCatchment for hospital {:4.0f} complete'.format(_thread_id), end="")
    return(_thread_id, [ polygon.copy(deep=True) for polygon in polygons ]) 

### measure_acc_par

Parallel implementation of accessibility measurement.

Args:

* hospitals: Geodataframe of hospitals
* pop_data: Geodataframe containing population data
* network: OSMNX street network
* distances: list of distances to calculate catchments for
* weights: list of floats to apply to different catchments
* num\_proc: number of processors to use.

Returns:

* Geodataframe of catchments with accessibility statistics calculated

In [ ]:
def hospital_acc_unpacker(args):
    return hospital_measure_acc(*args)

def measure_acc_par (hospitals, pop_data, network, distances, weights, num_proc = 4):
    catchments = []
    for distance in distances:
        catchments.append(gpd.GeoDataFrame())
    pool = mp.Pool(processes = num_proc)
    hospital_list = [ hospitals.iloc[i] for i in range(len(hospitals)) ]
    results = pool.map(hospital_acc_unpacker, zip(range(len(hospital_list)), hospital_list, itertools.repeat(pop_data), itertools.repeat(distances), itertools.repeat(weights)))
    pool.close()
    results.sort()
    results = [ r[1] for r in results ]
    for i in range(len(results)):
        for j in range(len(distances)):
            catchments[j] = catchments[j].append(results[i][j], sort=False)
    return catchments

### overlap_calc

Calculates and aggregates accessibility statistics for one catchment on our grid file.

Args:

* \_id: thread ID
* poly: GeoDataFrame representing a catchment area
* grid_file: a GeoDataFrame representing our grids
* weight: the weight to applied for a given catchment
* service_type: the service we are calculating for: ICU beds or ventilators

Returns:

* Tuple containing:
    * thread ID
    * Counter object (dictionary for numbers) with aggregated stats by grid ID number

In [ ]:
from collections import Counter
def overlap_calc(_id, poly, grid_file, weight, service_type):
    value_dict = Counter()
    if type(poly.iloc[0][service_type])!=type(None):           
        value = float(poly[service_type])*weight
        intersect = gpd.overlay(grid_file, poly, how='intersection')
        intersect['overlapped']= intersect.area
        intersect['percent'] = intersect['overlapped']/intersect['area']
        intersect=intersect[intersect['percent']>=0.5]
        intersect_region = intersect['id']
        for intersect_id in intersect_region:
            try:
                value_dict[intersect_id] +=value
            except:
                value_dict[intersect_id] = value
    return(_id, value_dict)

def overlap_calc_unpacker(args):
    return overlap_calc(*args)

### overlapping_function

Calculates how all catchment areas overlap with and affect the accessibility of each grid in our grid file.

Args:

* grid_file: GeoDataFrame of our grid
* catchments: GeoDataFrame of our catchments
* service_type: the kind of care being provided (ICU beds vs. ventilators)
* weights: the weight to apply to each service type
* num\_proc: the number of processors

Returns:

* Geodataframe - grid\_file with calculated stats

In [ ]:
def overlapping_function (grid_file, catchments, service_type, weights, num_proc = 4):
    grid_file[service_type]=0
    pool = mp.Pool(processes = num_proc)
    acc_list = []
    for i in range(len(catchments)):
        acc_list.extend([ catchments[i][j:j+1] for j in range(len(catchments[i])) ])
    acc_weights = []
    for i in range(len(catchments)):
        acc_weights.extend( [weights[i]]*len(catchments[i]) )
    results = pool.map(overlap_calc_unpacker, zip(range(len(acc_list)), acc_list, itertools.repeat(grid_file), acc_weights, itertools.repeat(service_type)))
    pool.close()
    results.sort()
    results = [ r[1] for r in results ]
    service_values = results[0]
    for result in results[1:]:
        service_values+=result
    for intersect_id, value in service_values.items():
        grid_file.loc[grid_file['id']==intersect_id, service_type] += value
    return(grid_file)

### normalization

Normalizes our result (Geodataframe) for a given resource (res).

In [ ]:
def normalization (result, res):
    result[res]=(result[res]-min(result[res]))/(max(result[res])-min(result[res]))
    return result

### file_import

Imports all files we need to run our code and pulls the Illinois network from OSMNX if it is not present (will take a while). 

**NOTE:** even if we calculate accessibility for just Chicago, we want to use the Illinois network (or at least we should not use the Chicago network) because using the Chicago network will result in hospitals near but outside of Chicago having an infinite distance (unreachable because roads do not extend past Chicago).

Args:

* pop_type: population type, either "pop" for general population or "covid" for COVID-19 cases
* region: the region to use for our hospital and grid file ("Chicago" or "Illinois")

Returns:

* G: OSMNX network
* hospitals: Geodataframe of hospitals
* grid_file: Geodataframe of grids
* pop_data: Geodataframe of population

In [ ]:
def output_map(output_grid, base_map, hospitals, resource):
    ax=output_grid.plot(column=resource, cmap='PuBuGn',figsize=(18,12), legend=True, zorder=1)
    # Next two lines set bounds for our x- and y-axes because it looks like there's a weird 
    # Point at the bottom left of the map that's messing up our frame (Maja)
    ax.set_xlim([314000, 370000])
    ax.set_ylim([540000, 616000])
    base_map.plot(ax=ax, facecolor="none", edgecolor='gray', lw=0.1)
    hospitals.plot(ax=ax, markersize=10, zorder=1, c='blue')

### Run the model

Below you can customize the input of the model:

* Processor - the number of processors to use
* Region - the spatial extent of the measure
* Population - the population to calculate the measure for
* Resource - the hospital resource of interest
* Hospital - all hospitals or subset to check code

In [ ]:
import ipywidgets
from IPython.display import display

processor_dropdown = ipywidgets.Dropdown( options=[("1", 1), ("2", 2), ("3", 3), ("4", 4)],
    value = 4, description = "Processor: ")

place_dropdown = ipywidgets.Dropdown( options=[("Chicago", "Chicago"), ("Illinois","Illinois")],
    value = "Chicago", description = "Region: ")

population_dropdown = ipywidgets.Dropdown( options=[("Population at Risk", "pop"), ("COVID-19 Patients", "covid") ],
    value = "pop", description = "Population: ")

resource_dropdown = ipywidgets.Dropdown( options=[("ICU Beds", "hospital_icu_beds"), ("Ventilators", "hospital_vents") ],
    value = "hospital_icu_beds", description = "Resource: ")

hospital_dropdown =  ipywidgets.Dropdown( options=[("All hospitals", "hospitals"), ("Subset", "hospital_subset") ],
    value = "hospitals", description = "Hospital:")

display(processor_dropdown,place_dropdown,population_dropdown,resource_dropdown)

In [ ]:
# Create point geometries for the entire graph (ESSENTIAL FOR CALCULATING CATCHMENT AREAS W DIJKSTRA METHOD)
for node, data in G.nodes(data=True):
    data['geometry']=Point(data['x'], data['y'])

In [ ]:
%%time
# G, hospitals, grid_file, pop_data = file_import (population_dropdown.value, place_dropdown.value)
G = network_setting(G)
# Modify code to react to processor dropdown (got rid of file_import function)
if population_dropdown.value == "pop":
    pop_data = pop_centroid(atrisk_data, population_dropdown.value)
elif population_dropdown.value == "covid":
    pop_data = pop_centroid(covid_data, population_dropdown.value)
# Modify code to react to processor dropdown
if hospital_dropdown.value == "hospitals":
    hopitals = hospitals = hospital_setting(hospitals, G)
elif hospital_dropdown.value == "hospital_subset":
    hopitals = hospitals = hospital_setting(hospital_subset, G)
hospitals = hospital_setting(hospitals, G)
distances=[10,20,30] # Distances in travel time
weights=[1.0, 0.68, 0.22] # Weights where weights[0] is applied to distances[0]
# Other weighting options representing different distance decays
# weights1, weights2, weights3 = [1.0, 0.42, 0.09], [1.0, 0.75, 0.5], [1.0, 0.5, 0.1]
resources = ["hospital_icu_beds", "hospital_vents"] # resources

#### Check OSM Network

compare to earlier speed counts

In [ ]:
%%time
## Get unique counts for each road network
# Turn nodes and edges in geodataframes
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)

# Count
print(edges['maxspeed_fix'].value_counts())
print(len(edges))

#### Code to show that catchment areas are properly calculated

allows us to see catchment areas of the subset selection

In [ ]:
# Check if functions are working properly
if hospital_dropdown.value == "hospital_subset":
    # Create point geometries for entire graph
    for node, data in G.nodes(data=True):
        data['geometry']=Point(data['x'], data['y'])
    
    # Create catchment
    poly = dijkstra_cca_polygons(G, hospitals['nearest_osm'][0], distances)
    
    # Reproject polygons
    for i in range(len(poly)):
        poly[i].crs = { 'init' : 'epsg:4326'}
        poly[i] = poly[i].to_crs({'init':'epsg:32616'})
        
    # Reproject hospitals 
    hospital_subset = hospital_subset.to_crs(epsg=32616)
    
    ## NEED TO PROJECT THE POLYGONS SO THAT THECAN BE COMPARED WITH HOSPITAL AND STREET NETWORK
    fig, ax = plt.subplots(figsize=(12,8))

    poly[0].plot(ax=ax, color="royalblue", label="10 min drive")
    poly[1].plot(ax=ax, color="cornflowerblue", label="20 min drive")
    poly[2].plot(ax=ax, color="lightsteelblue", label="30 min drive")
    
    hospital_subset.plot(ax=ax, color="red", legend=True, label = "hospital")
    
    # Add legend
    ax.legend()
    
else:
    None

In [ ]:
%%time
catchments = measure_acc_par(hospitals, pop_data, G, distances, weights, num_proc=processor_dropdown.value)

In [ ]:
%%time
for j in range(len(catchments)):
    catchments[j] = catchments[j][catchments[j][resource_dropdown.value]!=float('inf')]
result=overlapping_function (grid_file, catchments, resource_dropdown.value, weights, num_proc=processor_dropdown.value)

In [ ]:
%%time
result = normalization (result, resource_dropdown.value)

In [ ]:
result.head()

### Results & Discussion

to be written.

Continuous Accessibility Outputs

In [ ]:
%%time
hospitals = hospitals.to_crs({'init': 'epsg:26971'})
result = result.to_crs({'init': 'epsg:26971'})
output_map(result, pop_data, hospitals, resource_dropdown.value)

Classified Accessibility Outputs

### Conclusion

to be written.

### References

Luo, W., & Qi, Y. (2009). An enhanced two-step floating catchment area (E2SFCA) method for measuring spatial accessibility to primary care physicians. Health & place, 15(4), 1100-1107.